# json 파일에서 null, null인 부분 제거

In [1]:
import pandas as pd
import numpy as np
import glob
import json
import os
import gc
from tqdm import tqdm

In [2]:
def train_v2_json():
    # 원본 annotation파일 오픈
    with open('/data/ephemeral/home/datasets/jsons/origin_train.json', 'r') as train_json:
        origin_data = json.load(train_json)
    
    img_path_list = glob.glob('/data/ephemeral/home/datasets/images/train/*.jpg') # img list
    img_path_list.sort() # 확인하기 편하게 이름 순서로 정렬
    tbar = tqdm(img_path_list[3000:]) # 500개씩 끊어서 수행
    tmp_list = list()
    cnt = 0

    for img_path in tbar:
        cnt += 1
        img_name = img_path.split('/')[-1] # 이미지 이름 추출
        words = origin_data['images'][img_name]['words'] # 사진의 words 정보 가져오기
        tmp = dict() # 데이터 임시 저장
        # 모든 word 검사
        for word_id, word_data in words.items():
            ori = word_data.get('orientation')
            lan = word_data.get('language')
            # orientation, language 모두 None인 경우 특수문자로 보인다. -> 일반 문자만 저장할 예정
            if ori != None or lan != None:
                tmp[word_id] = word_data

        img_w = origin_data['images'][img_name]['img_w']
        img_h = origin_data['images'][img_name]['img_h']
        tmp_list.append({img_name: {'words': tmp, "img_w": img_w, "img_h": img_h}})

        if cnt % 100 == 0:
            with open('/data/ephemeral/home/datasets/jsons/train.json', 'r') as f:
                data = json.load(f)
            for current in tmp_list:
                key = list(current.keys())[0]
                data['images'][key] = current[key]
            with open('/data/ephemeral/home/datasets/jsons/train.json', 'w') as modified:
                json.dump(data, modified, indent=4)
        
    with open('/data/ephemeral/home/datasets/jsons/train.json', 'r') as f:
        data = json.load(f)
    for current in tmp_list:
        key = list(current.keys())[0]
        data['images'][key] = current[key]
    with open('/data/ephemeral/home/datasets/jsons/train.json', 'w') as modified:
        json.dump(data, modified, indent=4)

In [ ]:
train_v2_json()

100%|██████████| 272/272 [00:19<00:00, 13.75it/s]
